In [ ]:
import jax.numpy as jnp
from jax import random, grad, vmap
from functools import partial
import matplotlib.pyplot as plt
import jaxgp.regression as gpr
from jaxgp.kernels import RBF, Linear, Constant
import timeit
import time

# Full GPR

## Benchmark memory of fitmatrix and fitvector

## Benchmarking with a constant function

In [ ]:
data_sizes = [100, 200, 500, 1000, 2000, 3000, 4000, 5000]

kernel_1 = RBF()
kernel_2 = Linear()

### Timing of the training

In [ ]:
for elem in data_sizes:
    data_split = (elem, )

    X_data = jnp.linspace(0.0, 1.0, data_split[0]).reshape(-1,1)
    Y_data =jnp.ones(data_split[0]).reshape(-1,1)

    def test_RBF():
        model = gpr.ExactGPR(kernel=kernel_1, data_split=data_split, noise=0.1)
        model.train(X_data=X_data, Y_data=Y_data)

    print(f"N={elem}: {timeit.repeat(test_RBF, repeat=1, number=1)}")

### Timing of eval with constant X_grid for different training sizes

In [ ]:
# X = jnp.linspace(0.2, 0.8, 5000)

# for elem in data_sizes:
#     data_split = (elem, )

#     X_data = jnp.linspace(0.0, 1.0, data_split[0]).reshape(-1,1)
#     Y_data =jnp.ones(data_split[0]).reshape(-1,1)

#     model = gpr.ExactGPR(kernel=kernel_1, data_split=data_split, noise=0.1)
#     model.train(X_data=X_data, Y_data=Y_data)

#     def test_RBF():
#         model.eval(X, return_std=True)


#     print(f"N={elem}: {timeit.repeat(test_RBF, repeat=5, number=1)}")

# Sparse GPR

### Timing of the training

In [ ]:
for elem in data_sizes:
    data_split = (elem, )

    X_data = jnp.linspace(0.0, 1.0, data_split[0]).reshape(-1,1)
    Y_data =jnp.ones(data_split[0]).reshape(-1,1)
    X_ref = jnp.linspace(0.0,1.0,data_split[0]//20).reshape(-1,1)

    def test_RBF():
        model = gpr.SparseGPR(kernel=kernel_1, data_split=data_split, X_ref=X_ref, noise=0.1)
        model.train(X_data=X_data, Y_data=Y_data)

    print(f"N={elem}: {timeit.repeat(test_RBF, repeat=1, number=1)}")

# Try a weird function

In [ ]:
# key = random.PRNGKey(int(time.time()))

# def f(x, noise=0):
#     return jnp.sin(2*x**2 + 1/(x+0.5/jnp.pi))*x + 1.0 + x + noise*random.normal(key,x.shape)

# gradient = vmap(grad(f), in_axes=(0,None))

# def df(x, noise=0):
#     return gradient(x, noise) + noise*random.normal(key,x.shape)

# # def f(x, noise=0.0):
# #     temp = jnp.where(x<0, -x, x)
# #     return temp + noise*random.normal(key,x.shape)

# # def df(x, noise=0.0):
# #     temp = jnp.where(x<0, -1.0, 1.0)
# #     return temp + noise*random.normal(key,x.shape)

In [ ]:
# bounds = jnp.array([0.0, jnp.pi])
# # bounds = jnp.array([-1.0, 1.0])

# noise = 0.1

# length_scale = 1.0

# num_datapoints = 1
# datagrid = jnp.linspace(*bounds,num_datapoints)
# data_values = f(datagrid,noise)
# num_derivpoints = (20,)
# # derivative_grid = jnp.linspace(*bounds,num_derivpoints[0])
# derivative_grid = random.uniform(key, num_derivpoints, minval=bounds[0], maxval=bounds[1])
# derivative_values = df(derivative_grid,noise)
# data_split = (num_datapoints, ) + num_derivpoints

# X_data = jnp.concatenate((datagrid,derivative_grid)).reshape(-1,1)
# Y_data = jnp.concatenate((data_values,derivative_values)).reshape(-1,1)

# num_referencepoints = 33
# # referencegrid = jnp.linspace(*bounds,num_referencepoints).reshape(-1,1)
# referencegrid = random.uniform(key, (num_referencepoints,), minval=bounds[0], maxval=bounds[1]).reshape(-1,1)

# num_gridpoints = 101
# grid = jnp.linspace(*bounds,num_gridpoints)

# kernel = Constant() * RBF()# + Linear()
# kernel_params = (length_scale, length_scale)#, 1.0, 1.0)

In [ ]:
# model = gpr.ExactGPR(kernel,data_split,kernel_params,noise=noise)
# model.train(X_data,Y_data)
# (means_full, stds_full) = model.eval(grid.reshape(-1,1),True)
# means_full = means_full.reshape(-1)
# stds_full = stds_full.reshape(-1)

# model = gpr.SparseGPR(kernel,data_split,referencegrid,kernel_params,noise=noise)
# model.train(X_data,Y_data)
# (means_ppa, stds_ppa) = model.eval(grid.reshape(-1,1),True)
# means_ppa = means_ppa.reshape(-1)
# stds_ppa = stds_ppa.reshape(-1)

In [ ]:
# plt.plot(grid,means_full,label="full")
# plt.fill_between(grid,means_full-2*stds_full,means_full+2*stds_full,alpha=0.5)
# plt.plot(grid,means_ppa,label="ppa")
# plt.fill_between(grid,means_ppa-2*stds_ppa,means_ppa+2*stds_ppa,alpha=0.5)
# plt.plot(grid,f(grid,0.0),color="grey",ls="--",label="true function")
# plt.scatter(datagrid,data_values,color="r",label="function values")
# plt.scatter(derivative_grid,jnp.zeros_like(derivative_grid))
# plt.scatter(referencegrid.reshape(-1), -jnp.ones_like(referencegrid.reshape(-1)))
# plt.grid()
# plt.legend()